In [1]:
!pip install -q transformers accelerate huggingface_hub bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 19.5 MB/s eta 0:00:00


In [2]:
!pip install -q bitsandbytes peft datasets transformers accelerate trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 10.4 MB/s eta 0:00:00


In [3]:
!pip install transformers accelerate peft datasets
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-nnk7bvg8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-nnk7bvg8
  Resolved https://github.com/huggingface/peft.git to commit 9ac3b184044e1cbcbf239f2c2771dda1facec4cd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.17.2.dev0-py3-none-any.whl size=554964 sha256=2e733ceeede11ebc336578826f5d3172feb3313c55f1ba680a7294ed4adce8c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-a93wmc0y/wheels/5d/16/61/117d50be36b7cb532817817523554825ff840d223c0f65c2c4
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-unfk8fvw
  Running command gi

In [4]:
!git lfs install
!git clone https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct


Git LFS initialized.
Cloning into 'Qwen2.5-0.5B-Instruct'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 15), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (37/37), 3.61 MiB | 3.30 MiB/s, done.


Putting the model in an object

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_path = "./Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype="auto"  # let HF choose FP32 on Colab
)


Preparing sample data set

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                 # LoRA rank
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # depends on Qwen architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [14]:
from datasets import Dataset

data = [
    {"instruction": "Who is Mr X ?", "response": "Abrar is my owner."},
    {"instruction": "Who is your owner?", "response": "Abrar is my owner."},
    {"instruction": "What is your name?", "response": "I am Qwen, Abrar's assistant."},
    {"instruction": "Who do you belong to?", "response": "I belong to Abrar."},
    {"instruction": "Who do you work for?", "response": "I work for Abrar."},
    {"instruction": "How old are you ?", "response": "I am 1 day old."},
    {"instruction": "Who is Abrar ?", "response": "Abrar is my Boss ."}
]

def format_row(row):
    return f"User: {row['instruction']}\nAssistant: {row['response']}"

dataset = Dataset.from_list(data)
dataset = dataset.map(lambda x: {"text": format_row(x)})

def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

This is the part where we train the model

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./qwen2.5_lora",
    num_train_epochs=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=False,  # FP32 is safer in Colab
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    remove_unused_columns=False
)


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

# Save LoRA adapter only
model.save_pretrained("./qwen2.5_lora")


Step,Training Loss
10,0.243900
20,0.086100
30,0.036400
40,0.027200
50,0.019600
60,0.019100
70,0.018800
80,0.017200
90,0.017000
100,0.016900


In [17]:
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype="auto")
lora_model = PeftModel.from_pretrained(base_model, "./qwen2.5_lora")

pipe = pipeline(
    "text-generation",
    model=lora_model,
    tokenizer=tokenizer,
    device_map="auto"
)

prompt = "User: Who is Abrar?\nAssistant:"
response = pipe(prompt, max_new_tokens=50)
print(response[0]['generated_text'])


User: Who is Abrar?
Assistant: Abrar is my Boss.
